# 교차 검증과 그리드 서치

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
# 데이터를 데이터프레임화

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
# 알코올 도수, 당도, pH 열을 data에 저장
target = wine['class'].to_numpy()
# class 열을 target에 저장

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)
# 훈련 세트와 테스트 세트를 8:2로 나눔

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)
# train_input과 train_target을 train_test_split() 함수에 넣어서
# 검증 세트를 생성

In [5]:
print(sub_input.shape, val_input.shape)
# 훈련 세트와 검증 세트의 크기 확인

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
# DecisionTreeClassifier 클래스 import
dt = DecisionTreeClassifier(random_state=42)
# DecisionTreeClassifier 객체 dt를 생성
dt.fit(sub_input, sub_target)
# dt 훈련
print(dt.score(sub_input, sub_target))
# 훈련 세트 점수 출력
print(dt.score(val_input, val_target))
# 검증 세트 점수 출력

0.9971133028626413
0.864423076923077


## 교차 검증

In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
# cross_validate()에 인수로 평가 모델 객체, train_input, target_input을 전달
print(scores)
# scores에는 return 값으로 'fit_time', 'score_time', 'test_score' 키를 가진 딕셔너리 저장
# fit_time은 훈련하는 시간, score_Time은 검증하는 시간을 의미
# test_score는 검증 폴드의 점수
# 교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균내어 얻을 수 있음

{'fit_time': array([0.0138967 , 0.00946498, 0.01077986, 0.02826023, 0.02357745]), 'score_time': array([0.00164199, 0.00113106, 0.00137305, 0.00352311, 0.00163126]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
import numpy as np

print(np.mean(scores['test_score']))
# 교차 검증의 최종 점수 출력

0.855300214703487


In [9]:
from sklearn.model_selection import StratifiedKFold
# cross_validate() 함수는 분류 보델의 경우 StratifiedKFold를 사용
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
# cv=StratifiedKFold()를 추가함
print(np.mean(scores['test_score']))
# 교차 검증의 최종 점수 출력

0.855300214703487


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# 훈련 세트를 섞은 후 10-폴드 교차 검증 수행
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))
# 교차 검증의 최종 점수 출력

0.8574181117533719


## 하이퍼파라미터 튜닝

In [11]:
from sklearn.model_selection import GridSearchCV
# GridSearchCV() 클래스 import
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
# min_impurity_decrease 매개변수의 최적값을 알기 위해 params에 리스트를 value로 하는 딕셔너리 생성

In [12]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# GridSearchCV의 객체 gs를 생성하며 탐색 대상 모델과 params 변수를 인수로 전달
# n_jobs=-1로 시스템에 있는 모든 CPU 코어를 사용할 수 있도록 지정

In [13]:
gs.fit(train_input, train_target)
# gs 객체에 fit() 메서드를 호출하여 min_impurity_decrease로 값을 바꾸가며 5번 실행
# GridSearchCV의 cv 매개변수 기본값은 5로 min_impurity_decrease 값마다 5폴드 교차 검증을 수행
# 총 25번 수행

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [14]:
dt = gs.best_estimator_
# 25개의 모델 중에서 검증 점수가 가장 높은 모델이 best_estimator_ 속성에 저장
print(dt.score(train_input, train_target))
# dt 점수 출력

0.9615162593804117


In [15]:
print(gs.best_params_)
# 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장

{'min_impurity_decrease': 0.0001}


In [16]:
print(gs.cv_results_['mean_test_score'])
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score' 키에 저장
# 5번의 교차 검증으로 얻은 점수를 출력

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [17]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
# argmax() 함수를 통해 가장 큰 값의 인덱스를 추출하여 best_index에 저장
print(gs.cv_results_['params'][best_index])
# 출력하여 best_params_ 값과 일치하는지 확인

{'min_impurity_decrease': 0.0001}


In [18]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }
# 조금 더 복잡한 매개변수 조합 탐색
# min_impurity_decrease, max_depth, min_samples_split에 대해서 각각 9, 15, 10가지의 조합을 표현
# 기본 5-폴드 교차 검증을 수행하여, 9*15*10*5 = 6.750개의 모델의 조합을 시행

In [19]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# n_jobs=-1로 설정
gs.fit(train_input, train_target)
# 그리드 서치를 실행

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [20]:
print(gs.best_params_)
# 최상의 매개변수 조합 출력

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [21]:
print(np.max(gs.cv_results_['mean_test_score']))
# 최상의 교차 검증 점수 출력

0.8683865773302731


### 랜덤 서치

In [22]:
from scipy.stats import uniform, randint
# uniform과 randint 클래스 import

In [23]:
rgen = randint(0, 10)
# 0 ~ 10 사이의 범위를 갖는 randint 객체 생성
rgen.rvs(10)
# 10개의 숫자를 샘플링

array([9, 8, 7, 8, 7, 8, 5, 6, 1, 0])

In [24]:
np.unique(rgen.rvs(1000), return_counts=True)
# 1,000개의 숫자를 샘플링해서 각 숫자의 개수를 출력

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98, 112,  97,  99, 101,  99,  96,  90,  98, 110]))

In [25]:
ugen = uniform(0, 1)
# 0 ~ 1 사이의 범위를 갖는 uniform 객체 생성
ugen.rvs(10)
# 10개의 숫자를 샘플링

array([0.5996621 , 0.36612186, 0.39708277, 0.10997034, 0.2086858 ,
       0.48615767, 0.3466983 , 0.00087719, 0.8539231 , 0.06455077])

In [26]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }
# 탐색할 매개변수 딕셔너리 params 생성
# min_samples_leaf 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수
# 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않음

In [27]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
# 매개변수 n_iter을 통해 100번을 샘플링하여 교차 검증을 수행하여 최적의 매개변수 조합을 찾음

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ac5b5923640>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ac5b59218d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ac5b5923460>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ac5b5921780>},
                   random_state=42)

In [29]:
print(gs.best_params_)
# 최적의 매개변수 조합 출력

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [30]:
print(np.max(gs.cv_results_['mean_test_score']))
# 최고의 교차 검증 점수 출력

0.8695428296438884


In [31]:
dt = gs.best_estimator_
# 최적의 모델을 dt에 저장
print(dt.score(test_input, test_target))
# dt의 성능 점수 출력

0.86
